In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'

In [3]:
cap = cv2.VideoCapture('vid1_IR.avi')

In [6]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('IR', G)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()